# Distributions, priors, and basic sampling in CUQIpy


## <font color='blue'> Contents of this notebook: </font>
* [Introduction](#Introduction)
* [Learning objectives](#r-learning-objectives)
* [References](#References)
* [Distributions in CUQIpy](#Distributions-in-CUQIpy)
* [Multivariate Gaussian distribution](#Multivariate-Gaussian-distribution)
* [Elaboration on conditional distributions](#Conditional-distributions)
* [Markov random fields in CUQIpy](#Markov-Random-Field)
* [Using Geometry in distributions for parametrization and mapping](#Geometry)
* [User-defined distributions](#User-defined-distributions)
* [Elaboration on Geometry in distributions and samples](#Geometry-in-distributions-and-samples)



## <font color='blue'> Learning objectives: </font> <a name="r-learning-objectives"></a>
- Set up random variables following uni- or multivariate distributions in CUQIpy, generate samples, and visualize them.
- Set up conditional distributions in CUQIpy - simple and using lambda functions.
- ★ Create a user-defined distribution from a logpdf function.
- ★ Explain the use of Geometry in distributions and samples.




★ Indicates optional section.

**References:**

- [1] *Bardsley, Johnathan. 2018. Computational Uncertainty Quantification for Inverse Problems. SIAM, Society for Industrial and Applied Mathematics.*



In [ ]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import cuqi
from cuqi.distribution import Gaussian, GMRF
from cuqi.samples import Samples
from cuqi.geometry import Continuous1D, Image2D, StepExpansion, KLExpansion,\
    Discrete, MappedGeometry


## <font color='blue'> Distributions in CUQIpy  </font> <a class="anchor" id="Multivariate"></a> 
 


In the previous chapter, we saw a couple of univariate and bivariate `Gaussain` distributions. Let us now look at more general multivariate distributions.
CUQIpy currently implements a number of multivariate distributions in the `cuqi.distribution` module:

In [ ]:
[dist for dist in dir(cuqi.distribution) if not dist.startswith('_')]

and more can easily be added when needed.


## <font color='blue'> Multivariate Gaussian distribution </font> <a class="anchor" id="Multivariate-Gaussian-distribution"></a>

To demonstrate, we specify here a 4-element random variable `y` following a Gaussian distribution with independent elements:

$$\mathbf{y} \sim \mathrm{Gaussian}(\mu,\mathrm{diag}(\sigma^2)) \quad \text{for} \quad \mu = [5, 3, 1, 0]^T \quad \text{and} \quad \sigma = [1,2,3, 0.5]$$

In [ ]:
true_mu = np.array([5, 3, 1, 0])
true_sigma = np.array([1, 2, 3, 0.5])
y = cuqi.distribution.Gaussian(mean=true_mu, cov=true_sigma**2)

We can take a look at the distribution by printing it and some of its properties.

In [ ]:
print(y)
print(y.dim)
print(y.name)

**Note about the distribution name:**

- The distribution name is inferred automatically from the assignment statement `y = cuqi.distribution.Gaussian(mean=true_mu, cov=true_sigma**2)`. 
- Since the distribution was assigned to the variable `y`, by default, the distribution name will be `y`.
- One can specify a different name by explicitly passing the `name` argument, e.g. `y = cuqi.distribution.Gaussian(mean=true_mu, cov=true_sigma**2, name='y1')`.


We can query other information about this distribution such as its mean:

In [ ]:
print(y.mean)

and its diagonal covariance matrix (represented as a 1D array of the diagonal values):

In [ ]:
print(y.cov)

We generate a single sample which produces a 4-element `CUQIarray`:

In [ ]:
y.sample()

If we ask for more than one sample, say 1000, we get a `Samples` object with 1000 columns each holding a 4-element sample.

**note about the `Samples` object:** The `Samples` object is essentially an array in which each column contains one sample, and further equipped with a number of methods for computing samples statistics, diagnostics and plotting.

In [ ]:
y_samples = y.sample(1000)
print(y_samples)
y_samples.shape

We can look at all the plotting methods available for the `Samples` object:

In [ ]:
# A list of all plotting methods
[method for method in dir(y_samples) if method.startswith('plot')] 

We can plot chains of, for example, 2 of the 4 variable samples, here we pick element 2 and 0:

In [ ]:
y_samples.plot_chain([2, 0])

As well as plot a few individual 4-element samples:

In [ ]:
y_samples.plot();

By default 5 random samples are plotted, but we can also specify indices of specific samples we wish to plot, like every 100th sample:

In [ ]:
y_samples.plot([0, 100, 200, 300, 400, 500, 600, 700, 800, 900]);

We can also plot the sample mean and compare with the true mean of the distribution:

In [ ]:
y_samples.plot_mean(label="Sample mean")
plt.plot(y.mean, 'o', label="Distribution mean")
plt.legend()

##### <font color='magenta'>Exercise:</font>
1. Plot mean with 90% credibility interval, hint: `help(y_samples.plot_ci)`.
2. Include in the credibility interval plot a comparison with the true mean using the `exact` keyword argument of `plot_ci`.
3. ★ Experiment with other plotting methods available for the `Samples` object.


In [ ]:
# your code here



## <font color='blue'> Geometry object: representation, parametrization and mapping </font> <a class="anchor" id="Geometry"></a>


A single sample from a distribution (in vector form) can be interpreted in various ways. It can be, for example, a vector of function values at 1D or 2D grid points, an image, a vector of expansion coefficients, or a collection of variables, e.g. the temperature measurement at four cities: A, B, C, and D. 

In CUQIpy, the [`cuqi.geometry`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.geometry.html) module provides classes for different representations of a vector, e.g `Continuous1D`, `Continuous2D`, `Image2D`, and `Discrete`.     


Here we explore geometry assignment for `Distributions`, `Samples` and `CUQIarray`.

##### <font color='magenta'> Geometry are used to specify variable representation </font>
First let us, again, create 1000 samples from the 4-element distribution $y$ we created earlier:

In [ ]:
y_samples = y.sample(1000)

If no geometry is provided by the user, when creating a `Distribution` for example, CUQIpy will assign `_DefaultGeometry1D` (trivial geometry) to the distribution and the `Samples` produced from it.

In [ ]:
print(y.geometry)
print(y_samples.geometry)

As we saw, samples are plotted with line plot by default, which is due to how the `_DefaultGeometry` interprets the samples. 

In [ ]:
y_samples.plot([100,200,300])

We may equip the distribution with a different geometry, either when creating it, or afterwards. Let us for example assign an `Image2D` geometry to the distribution `y`. First we create the `Image2D` geometry and we assume the shape of the image is $2\times2$:

In [ ]:
geom_image = Image2D((2,2))
print(geom_image)

We can check the number of parameters (parameter dimension) of the geometry:

In [ ]:
geom_image.par_dim

We can also check the shape of its representation (size of the image in this case) using the property `fun_shape`:

In [ ]:
geom_image.fun_shape

Now we equip the distribution `y` with this `Image2D` geometry.

In [ ]:
y.geometry = geom_image

Check the geometry of `y`:

In [ ]:
y.geometry

With this distribution set up, we are ready to generate some samples

In [ ]:
# call method to sample
y_samples = y.sample(50)

We can check that we have produced 50 samples, each of size 4:

In [ ]:
y_samples.shape

We plot a couple of samples:

In [ ]:
y_samples.plot()   

What if the 4 parameters in samples have a different meaning? For example, the four parameters might represent labelled quantities such as temperature measurement at four cities A, B, C, D. In this case, we can use a `Discrete` geometry:

In [ ]:
geom_discrete = Discrete(['Temperature A', 'Temperature B', 'Temperature C', 'Temperature D'])
print(geom_discrete)

We can update the distribution's geometry and generate some new samples:

In [ ]:
y.geometry = geom_discrete

In [ ]:
y_samples = y.sample(100)

The samples will now know about their new `Discrete` geometry and the plotting style will be changed:

In [ ]:
y_samples.plot();

The credibility interval plot style is also updated to show errorbars for the `Discrete` geometry:

In [ ]:
y_samples.plot_ci(95, exact=true_mu)

And similarly, in the chain plot, the legend reflects the particular labels:

In [ ]:
y_samples.plot_chain([2,0])

##### <font color='magenta'> Geometry are used to parameterize the variables </font>

In CUQIpy we have geometries that represents a particular parameterization of the variables (e.g. `StepExpansion`, `KLExpansion`, etc)
 For example, in the `StepExpansion` geometry, the parameters represent expansion coefficients for equidistance-step basis functions. You can read more about `StepExpansion` by typing `help(cuqi.geometry.StepExpansion)` in a the code cell below or by looking at the [documentation of `StepExpansion`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.geometry/cuqi.geometry.StepExpansion.html)

Let us create a `StepExpansion` geometry, we first need to create a grid on which the step functions are defined, let us assume the grid is defined on the interval $[0,1]$:



In [ ]:
grid = np.linspace(0, 1, 100)

Then we create the `StepExpansion` geometry with 4 steps and assign it to the distribution `y`:

In [ ]:
geom_step_expantion = StepExpansion(grid, n_steps=4)
y.geometry = geom_step_expantion
print(y.geometry)


Let us samples the distribution `y` and plot the samples:

In [ ]:
y_step_samples = y.sample(100)
y_step_samples.plot()

Note that the samples are now represented as expansion coefficients of the step functions.

Examples of using `StepExpansion` and `KLExpansion` geometries is in the context of a PDE parameterization for a heat 1D BIP can be found [here](https://github.com/CUQI-DTU/Paper-CUQIpy-2-PDE/blob/main/heat_1D/heat_1D_part1.ipynb), and [here](https://github.com/CUQI-DTU/Paper-CUQIpy-2-PDE/blob/main/heat_1D/heat_1D_part2.ipynb) {cite}`Alghamdi_2024`, respectively.

##### <font color='magenta'> Geometry are used to map the variables </font>

In `CUQIpy`, we provide the `MappedGeometry` object which equips geometries with a mapping function that are applied to the variables, this mapping can also be viewed as parametrization. An example of commonly used mapping in inverse problems is $e^x$ for unknown parameters $x$ to ensure positivity regardless of the value of $x$.

Let us use the `MappedGeometry` to map the `StepExpansion` geometry we created earlier with the exponential function:

In [ ]:
geom_step_expantion_mapped = MappedGeometry(geom_step_expantion, map=lambda x: np.exp(x))

Let us again, assign the `MappedGeometry` to the distribution `y` and generate samples and plot them:

In [ ]:
y.geometry = geom_step_expantion_mapped
y_mapped_step_samples = y.sample(100)
y_mapped_step_samples.plot()


Note that the samples are still representing the expansion coefficients of the step functions, but the mapping function $e^x$ has been applied to the samples. All samples are now non-negative.

##### <font color='magenta'> ★ Elaboration: Specifying Geometry object for CUQIarray:</font>
Geometries can also be specified for `CUQIarray`, the basic array structure in CUQIpy. Let us create a `CUQIarray` object with four parameters as follows: 

In [ ]:
q = cuqi.array.CUQIarray([1,5,6,0])

We look at the geometry property 

In [ ]:
q.geometry

And then let us plot our variable `q`:

In [ ]:
q.plot()

We now choose a different interpretation for the variable `q` by changing its geometry to, for example, the `Image2D` geometry we created, and then we plot:

In [ ]:
q.geometry = geom_image
q.plot()

Finally we set the `Discrete` geometry we created as the geometry for `q` and plot: 

In [ ]:
q.geometry = geom_discrete
q.plot()

## <font color='blue'> Markov random fields in CUQIpy </font> <a class="anchor" id="Markov-Random-Field"></a>

In some cases, we may want to generate samples that represent a field with some spatial correlation and smoothness properties.

One `CUQIpy` distribution object that can be achieved for this purpose is the Gaussian Markov random field (GMRF) distribution. This distribution assumes a Gaussian distribution on the differences between neighboring elements of $\mathbf{x}$, i.e. in 1D:

\begin{align*}
x_i - x_{i-1} \sim \mathrm{Gaussian}(0, d^{-1}), \quad i=1, \ldots, n,
\end{align*}

where we purposely leave out the details on the boundary conditions for this notebook.

To simplify the notation, we denote by *GMRF* the distribution that induces this property on a vector $\mathbf{x}$ defined by its mean and precision $d$. That is, the above can be written as

\begin{align*}
\mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d),
\end{align*}


with some choice of the precision say $d=50$. For more details on GMRF see the first CUQIpy paper [2].

The GMRF distribution is implemented in CUQIpy as [GMRF class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.GMRF.html#cuqi.distribution.GMRF) and can be used as follows:



In [ ]:
# Define prior precision
d = 50
n = 200
# Define GMRF prior (zero boundary conditions are the default)
x_GMRF = GMRF(np.zeros(n), d)

##### <font color='magenta'>Exercise:</font>

1. Can you generate and plot a realization (sample) of `x_GMRF`? Does the realization show spatial correlation?
2. Create a Gaussian distribution `x_Gaussian` with mean `np.zeros(n)` and precision `50`, and compare a sample from the GMRF distribution with the Gaussian distribution by plotting them on the same plot. What do you observe?
3. ★ Generate 100000 samples of `x_GMRF` and store them in variable `x_GMRF_samples`. Verify the following about the distribution of the differences. Focus only on verifying the difference between elements 30 and 31 as a representative example.
    - The mean of the difference between elements 30 and 31 is close to 0.
    - The variance of the difference between elements 30 and 31 is close to $1/50$.
    - Hint: Use this line to create a `Samples` object of the differences `diff_30_31_samples = Samples((x_GMRF_samples.samples[31] - x_GMRF_samples.samples[30]).reshape(1, -1))`.

In [ ]:
# your code here


##### <font color='magenta'>Other Markov random fields in CUQIpy:</font>
- Cauchy Markov Random Field (CMRF): [CMRF class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.CMRF.html#cuqi.distribution.CMRF)
- Laplace Markov Random Field (LMRF): [LMRF class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.LMRF.html#cuqi.distribution.LMRF)

`CMRF` and `LMRF` are similar to `GMRF` but with different distributions on the differences between neighboring elements in the signal, where `CMRF` assumes a Cauchy distribution and `LMRF` assumes a Laplace distribution. `LMRF` and `CMRF` are particularly useful in cases in which the signal to be inferred has sharp edges (jumps).

This [1D deconvolution example](https://github.com/CUQI-DTU/Paper-CUQIpy-1-Core/blob/main/deconvolution1D/paper1_deconv1D_square.ipynb) from {cite}`Riis_2024`  illustrates and compares using the three Markov random fields in a 1D  problem.

## <font color='blue'> ★ Elaboration on conditional distributions </font> <a class="anchor" id="Conditional"></a> 

CUQIpy also support conditional probability distributions, which are probability distributions that are defined conditionally on the value of one or more other random variables. We saw in chapter 1, an example of conditional distribution. The data distribution `b` conditioned on input `x`.

In CUQIpy defining conditional distributions is simple. Assume we are interested in defining the Normal distribution conditioned on a random variable representing the standard deviation, e.g.

$$ z \mid \mathrm{std} \sim \mathrm{Normal}(0,\mathrm{std}^2) $$

This can simply be achieved by *omitting* the keyword argument for the standard deviation, when defining the distribution, as shown in the following code

In [ ]:
z = cuqi.distribution.Normal(mean=0)

Printing it will tell us that the variable `std` has not been specified, i.e., it is a *conditioning variable*:

In [ ]:
print(z)

Because $z$ is a conditional distribution, we cannot evaluate the logpdf or sample it directly without specifying the value of the conditioning variable (the standard deviation in this case). Hence this code will fail to run:

In [ ]:
# This code will give an error so we wrap it in a try/except block and print the error
try:
    z.sample()
except Exception as e:
    print(e)

To specify the conditioning variable we use the "call" syntax, i.e., `z(std=2)` to set the value of the standard deviation in the conditional distribution as shown below.

In [ ]:
z(std=2).sample()

In fact, conditioning creates a new *unconditional* distribution. Here printing reveals that it does not have any conditioning variables:

In [ ]:
z_given_std = z(std=2)
print(z_given_std)

We expect we can then sample it directly, which is confirmed:

In [ ]:
z_given_std.sample()

In general one may need more flexibility than simply conditioning directly on the attributes of the distribution. Let us assume we want to define a distribution that is conditional on the variance - denoted $d$ - rather than the standard deviation of the normal distribution, i.e.

$$ w \mid d \sim \mathrm{Normal}(0,d). $$

In CUQIpy this can be achieved through *lambda* functions. A lambda function is the Python equivalent of a MATLAB anonymous function, i.e. a function defined in a single line with the following syntax for an example function the simply sums two input arguments:

In [ ]:
myfun = lambda v1, v2: v1+v2

In [ ]:
myfun(5,7)

We can pass a lambda function directly as an argument to a CUQIpy distribution, e.g.,

In [ ]:
w = cuqi.distribution.Normal(mean=0, std=lambda d: np.sqrt(d))
print(w)

where we see that `d` is now the conditioning variable instead of `std` as before.

We can then pass a value for `d` to condition on, which allows us to sample from the now fully specified distribution:

In [ ]:
w(d=2).sample()

What actually happens behind the scenes is that writing `w(d=2)` defined a new CUQIpy distribution, where the standard deviation is set by evaluating the lambda function. This can be seen by storing the new distribution as follows.

In [ ]:
w_given_d = w(d=2)
w_given_d.std

This framework allows for a lot of flexibility in defining conditional distributions. For example we can define lambda functions for all attributes (here the mean and the standard deviation):

In [ ]:
#Functions for mean and std with various (shared) inputs
mean = lambda sigma, gamma: sigma+gamma
std  = lambda delta, gamma: np.sqrt(delta+gamma)

u = cuqi.distribution.Normal(mean, std)
print(u)

The three variable names `sigma`, `gamma` and `delta` used to define the two lambda functions for the mean and standard deviation are now the conditioning variables of the conditional distribution `u`.

By providing values for all three variables we obtain a fully specified distribution

In [ ]:
u_given_all = u(sigma=3, delta=5, gamma=-2)
print(u_given_all)

that we can sample:

In [ ]:
u_given_all.sample()

Conditional distributions play a major role when specifying Bayesian inverse problems in CUQIpy and in particular those problems that include Bayesian hierarchical models where some random variables depend on other random variables. We revisit this later in this mini-book.

## <font color='blue'> ★ User-defined distributions  </font> <a class="anchor" id="Userdefined"></a> 

In addition to the distributions provided by CUQIpy, there is also the possibility for users to specify new distributions. One option is to write their own class in the same style as existing distributions such as the Cauchy distribution (see code here: [Cauchy](https://github.com/CUQI-DTU/CUQIpy/blob/main/cuqi/distribution/_cauchy.py)).

Another option is to specify a user-defined distribution, which is convenient if one, for example, only wishes to evaluate the logpdf.

The example below demonstrates how to manually specify a normal distribution through a lambda function for the logpdf and compare it to the normal distribution defined in the beginning of this notebook.

We specify variables for the mean and the standard deviation and specify the lambda function for the logpdf. 

In [ ]:
mu1 = 0
std1 = 1

logpdf_func = lambda xx: -np.log(std1*np.sqrt(2*np.pi))-0.5*((xx-mu1)/std1)**2

To set up the user-defined distribution we need to specify the logpdf as well as its dimension (number of variables) since that cannot be automatically inferred from the lambda function:

In [ ]:
x_user = cuqi.distribution.UserDefinedDistribution(dim=1, logpdf_func=logpdf_func)

We can now evalute the logpdf, as well as the pdf:

In [ ]:
print(x_user.logpdf(0))
print(x_user.pdf(0))

We can compare this with the normal distribution from the beginning of the notebook and observe that their pdfs agree:

In [ ]:
plt.plot(grid, [x.pdf(node_k) for node_k in grid], label='CUQIpy Normal')
plt.plot(grid, [x_user.pdf(node_k) for node_k in grid], '--', label='User-defined Normal')
plt.legend()

We cannot sample the user-defined distribution because we have only provided the logpdf:

In [ ]:
try:
    x_user.sample()
except Exception as e:
    print(e)

We can equip the user-defined distribution with a sample_func which specified how to sample (it is up to the user to ensure consistency between logpdf and sample_func):

In [ ]:
x_user.sample_func = lambda : np.array(mu1 + std1*np.random.randn())

In [ ]:
x_user.sample()

We can compare the samples obtained from the original normal distribution and the user-defined:

In [ ]:
x_samples = x.sample(10000)

In [ ]:
x_user_samples = x_user.sample(10000)

We plot their histograms and note that they appear similar:

In [ ]:
x_samples.hist_chain(0,bins=100)
x_user_samples.hist_chain(0,bins=100)
plt.legend(['CUQIpy Normal', 'User-defined Normal'])